# Libraries

In [1]:
###
### TEAM 80 - OMNIVIDA
### ETL form Omnivida to Excel cleanned files to use
###

# Libraries
import pandas as pd
import datetime as dt
import numpy as np
import os
import re
import unidecode  # to remove accent in column names

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

#import generic_funcions as gf

# Functions definitions

In [2]:
archivos = os.listdir("./data/")

In [3]:
for namefile in archivos:
    print(namefile)

ACT.xlsx
ACT_DESAGREGADO.xlsx
Adherencia.xlsx
Antecedentes_familiares.xlsx
Antecedentes_patologicos.xlsx
Ayudas_diagnosticas.xlsx
Biologicos Asma.xlsx
Calidad de vida relacioada en salud.xlsx
Datos basicos.xlsx
Disnea.xlsx
Farmacovigilancia RAM.xlsx
Habitos.xlsx
Hospitalizaciones.xlsx
Incosistencias en reclamacion.xlsx
Medicamentos.xlsx
Mediciones de peso y talla.xlsx
Urgencias.xlsx
Vacunacion.xlsx


## Function: clean_columns

In [4]:
# clean_columns: remove accent, special characters and perform lower case
#                taken into account special files or columns as dates
# Parameters: df: a data frame of a file
# Value: return the df with names columns normalized (lowercase, no spetial characters, space by underscore)
#        the ID by patient is replaced by patient_id


def clean_columns(df):

    # -------------------------------------------------------- #
    # Search for name used to identify id patient if new file
    # are added this list must be updated
    # -------------------------------------------------------- #
    id_names = [
        'id', 'Id', 'ID', 'NUMERO IDENTIFICACION', 'ds_identificacion',
        'DS_IDENTIFICACION', 'Numero_Identificacion', 'Identificacion',
        'NRO_IDENTIFICACION', 'DS_IDENTIFICACION', 'IDENTIFICACIÓN',
        'Numero_de_documento'
    ]
    # Extract the name used un the file to identify the patient
    id_column = list(set(id_names) & set(df.columns))
    if len(id_column) == 0:
        # By default the first column
        df.rename(columns={df.columns[0]: "patient_id"}, inplace=True)
    else:
        # If there is a match use it
        df.rename(columns={id_column[0]: "patient_id"}, inplace=True)

    # Remove latin accent
    df.columns = list(map(lambda x: unidecode.unidecode(x), df.columns))
    # Replace non-printable spaces by simple space
    df.columns = df.columns.str.replace(pat="[\b]", repl=" ", regex=True)
    # Replace punctuation (like parenthesis) by simple space
    df.columns = df.columns.str.replace(pat="[^a-zA-Z0-9]",
                                        repl=" ",
                                        regex=True)
    # To lower case
    df.columns = df.columns.str.lower()
    # Strip blank space (delete extra spaces)
    df.columns = df.columns.str.strip()
    # Replace bank space by underscore
    df.columns = df.columns.str.replace(pat=" ", repl="_", regex=True)
    # Strip double underscore
    df.columns = df.columns.str.strip('_')

    # -------------------------------------------------------- #
    # Particular cases where dates are integers in Excel
    # -------------------------------------------------------- #

    # Error DATE type in "Hospitalizaciones.xlsx"
    if ('fecha_ingreso' in df.columns):
        if df.dtypes['fecha_ingreso'] != 'datetime64[ns]':
            df['fecha_ingreso'] = pd.TimedeltaIndex(
                df['fecha_ingreso'], unit='d') + dt.datetime(1899, 12, 30)
    if ('fecha_egreso' in df.columns):
        if df.dtypes['fecha_egreso'] != 'datetime64[ns]':
            df['fecha_egreso'] = pd.TimedeltaIndex(
                df['fecha_egreso'], unit='d') + dt.datetime(1899, 12, 30)

    # Error DATE type in Datos Basicos
    if ('fe_nacimiento' in df.columns):
        if (df.dtypes['fe_nacimiento'] != 'datetime64[ns]'):
            df['fe_nacimiento'] = pd.to_datetime(df['fe_nacimiento'])

    # Error DATE type in incosistencias_en_reclamacion
    if ('fe_registro' in df.columns):
        if (df.dtypes['fe_registro'] != 'datetime64[ns]'):
            df['fe_registro'] = pd.to_datetime(df['fe_registro'])

    # Put the patient_id in the first column
    dfcol = df['patient_id']
    df.drop(labels=['patient_id'], axis=1, inplace=True)
    df.insert(0, 'patient_id', dfcol)

    return df

## Function: SQLType

In [5]:
# SQLType: return SQL type variable given python type Useful to CREATE TABLE
# Parameters: string type
# Value: return SQL equivalent type
def SQLType(str):
    if str == 'int64':
        return "INTEGER"
    elif str == 'object':
        return "TEXT"
    elif str == 'datetime64[ns]':
        return "DATE"
    elif str == 'float64':
        return "FLOAT"
    else:
        return "TEXT"

## Funtion: CREATE_SQL_Script

In [6]:
# CREATE_SQL_Script: using the file name of a file write the script to CREATE TABLE
#                    in a SQL data base.
# Parameters: the file name and the data frame and the text file name (append mode)
# Value: write a text file with SQL script


def CREATE_SQL_Script(namefile, df_datatypes, ScriptName):
    # Clean the name of the file to be used as table name
    namefile_clean = namefile.replace(".xlsx", "").lower().replace(' ', '_')
    QSL_File = open(ScriptName, "a")
    QSL_File.write("CREATE TABLE " + namefile_clean + '(\n')

    # for datos_basicos the primary key is the patient_id for all others files is a regular consecutive number
    if namefile_clean != "datos_basicos":
        # Standard Primary key
        QSL_File.write(namefile_clean +
                       "_id INTEGER NOT NULL AUTO_INCREMENT PRIMARY KEY,\n")

        # loop over datatype that has the type of each column
        for id_df in range(len(df_datatypes) - 1):
            QSL_File.write(df_datatypes.loc[id_df, 'Variable'] + ' ' +
                           df_datatypes.loc[id_df, 'SQLType'] + ',\n')

        # The last out of the loop to close the parenthesis and make new lines
        id_df = id_df + 1
        QSL_File.write(df_datatypes.loc[id_df, 'Variable'] + ' ' +
                       df_datatypes.loc[id_df, 'SQLType'] + ',\n')
        QSL_File.write(
            'FOREIGN KEY (patient_id) references datos_basicos(patient_id)' +
            ');\n\n')
    else:
        # to run the script to create Tables the first must by datos_basicos
        for id_df in range(len(df_datatypes) - 1):
            if id_df == 0:
                # the first column is the patient_id primary key
                QSL_File.write(df_datatypes.loc[id_df, 'Variable'] + ' ' +
                               "INTEGER NOT NULL PRIMARY KEY" + ',\n')
            else:
                QSL_File.write(df_datatypes.loc[id_df, 'Variable'] + ' ' +
                               df_datatypes.loc[id_df, 'SQLType'] + ',\n')

        id_df = id_df + 1
        QSL_File.write(df_datatypes.loc[id_df, 'Variable'] + ' ' +
                       df_datatypes.loc[id_df, 'SQLType'] + ');\n\n')

    QSL_File.close()

## Function: CREATE_Copy_Script

In [7]:
# CREATE_Copy_Script: using the file name  create the scrip to copy data to DBase
# Parameters: the file name and the data frame and the text file name (append mode)
# Value: write a text file with copy script
def CREATE_Copy_Script(namefile, df_datatypes, ScriptName):
    # Clean the name of the file to be used as table name
    namefile_clean = namefile.replace(".xlsx", "").lower().replace(' ', '_')
    QSL_File = open(ScriptName, "a")

    QSL_File.write("%%sql postgresql://\n" + "\copy " + namefile_clean +
                   " FROM './data_clean/csv/" + namefile_clean + ".csv'" +
                   " with (format CSV, header true, delimiter ',');\n\n")

    QSL_File.close()

# Example

In [8]:
# Example
#namefile = archivos[12]
namefile = archivos[8]

df = pd.read_excel("./data/" + namefile)
df.info()
df = clean_columns(df)

df.drop_duplicates("patient_id", keep='last', inplace=True)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888 entries, 0 to 887
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   888 non-null    int64  
 1   edad                 887 non-null    float64
 2   genero               886 non-null    object 
 3   escolaridad          877 non-null    object 
 4   EstadoCivil          878 non-null    object 
 5   Preferencia          202 non-null    object 
 6   Departamento         886 non-null    object 
 7   Ciudad               886 non-null    object 
 8   Estrato              887 non-null    float64
 9   Zona                 887 non-null    object 
 10  AcompaÃ±ante         861 non-null    object 
 11  NivelSocioEconomico  871 non-null    object 
 12  Ocupacion            861 non-null    object 
 13  Regimen              861 non-null    object 
 14  TipoAfiliacion       861 non-null    object 
 15  RelacionLaboral      861 non-null    obj

,patient_id,edad,genero,escolaridad,estadocivil,preferencia,departamento,ciudad,estrato,zona,acompaa__ante,nivelsocioeconomico,ocupacion,regimen,tipoafiliacion,relacionlaboral,fe_nacimiento
0,500547,58.0,M,SECUNDARIA,CASADO (A),HETEROSEXUAL,ANTIOQUIA,MEDELLIN,3.0,URBANA,PACIENTE,A,EMPLEADO,CONTRIBUTIVO,COTIZANTE,EMPLEADO,1962-05-19
1,500588,15.0,M,PRIMARIA,SOLTERO (A),NaN,CUNDINAMARCA,BOGOTA,2.0,URBANA,MADRE,B,ESTUDIANTE,CONTRIBUTIVO,BENEFICIARIO,MENOR NO LABORA,2004-12-19
2,502989,29.0,M,UNIVERSITARIO,SOLTERO (A),HETEROSEXUAL,ANTIOQUIA,MEDELLIN,3.0,URBANA,MADRE,A,SIN DEFINIR,CONTRIBUTIVO,BENEFICIARIO,SIN EMPLEO,1990-08-22
3,506808,11.0,M,EDAD PREESCOLAR,SOLTERO (A),HETEROSEXUAL,ANTIOQUIA,BELLO,1.0,URBANA,MADRE,A,ESTUDIANTE,CONTRIBUTIVO,BENEFICIARIO,MENOR NO LABORA,2008-10-06
4,512938,22.0,F,SECUNDARIA,SOLTERO (A),HETEROSEXUAL,ANTIOQUIA,MEDELLIN,3.0,URBANA,PACIENTE,A,EMPLEADO,CONTRIBUTIVO,COTIZANTE,EMPLEADO,1997-10-15


# Run Script main

In [10]:
# Because our data are small files but Creating Excels is not needed
# Just for fun!

# To create and excel file with all the cleanned dataframes as sheet's
XLSwriter_Data = pd.ExcelWriter('./data_clean/All_Files_in_ONE.xlsx')

# To create and excel file with df.info() of each data frame as documentation
XLSwriter_Info = pd.ExcelWriter('./data_clean/Info_Files_in_ONE.xlsx')

# Becuase in some files there are patients' ids there are not in datos basicos
# We need to drop thoses rows. We will always merge with datos basicos
df_datos_basicos = pd.read_excel("./data/Datos basicos.xlsx")
# We only need the id column
df_datos_basicos = df_datos_basicos.loc[:, ['ID']]
df_datos_basicos.columns = ['patient_id']
df_datos_basicos.drop_duplicates(keep='last')

for namefile in archivos:
    # Read a file
    df = pd.read_excel("./data/" + namefile)
    # Clean dataframe
    df = clean_columns(df)
    df = pd.merge(df, df_datos_basicos, on='patient_id')

    # A clean name to save files
    namefile_clean = namefile.replace(".xlsx", "").lower().replace(' ', '_')

    # Remove duplicates en datos basicos
    if namefile_clean == "datos_basicos":
        df.drop_duplicates("patient_id", keep='last', inplace=True)
    else:
        df.drop_duplicates(keep='last', inplace=True)
    # Write each dataframe to a different worksheet.
    df.to_excel(XLSwriter_Data, sheet_name=namefile_clean[0:30], index=False)

    # Files also written in a new folder
    df.to_excel("./data_clean/xlsx/" + namefile_clean + ".xlsx", index=False)
    df.to_csv("./data_clean/csv/" + namefile_clean + ".csv", index=False)

    # Extract type variables to create a documentation file or SQL CREATE TABLE
    df_datatypes = pd.DataFrame(df.dtypes).reset_index()
    df_datatypes.columns = ['Variable', 'PythonType']
    df_datatypes['SQLType'] = list(map(SQLType, df_datatypes['PythonType']))

    # datos_basicos must be created first because it has the patient_id key
    # A different script can be used to be runned first
    if namefile_clean == 'datos_basicos':
        CREATE_SQL_Script(
            namefile, df_datatypes,
            "./data_clean/SQL_Script_CREATE_BD_datos_basicos.sql")
    else:
        CREATE_SQL_Script(namefile, df_datatypes,
                          "./data_clean/SQL_Script_CREATE_BD_All_others.sql")

    # Write each info as a sheet in Excel
    df_datatypes.to_excel(XLSwriter_Info,
                          sheet_name=namefile.lower()[0:30],
                          index=False)

    # Create the copy Script to be used in pstgress
    CREATE_Copy_Script(namefile, df_datatypes,
                       "./data_clean/SQL_Copy_Script.sql")

    print(" # ---------------- " + namefile + "-------------- #")

# Save Excels
XLSwriter_Data.save()
XLSwriter_Info.save()

 # ---------------- ACT.xlsx-------------- #
 # ---------------- ACT_DESAGREGADO.xlsx-------------- #
 # ---------------- Adherencia.xlsx-------------- #
 # ---------------- Antecedentes_familiares.xlsx-------------- #
 # ---------------- Antecedentes_patologicos.xlsx-------------- #
 # ---------------- Ayudas_diagnosticas.xlsx-------------- #
 # ---------------- Biologicos Asma.xlsx-------------- #
 # ---------------- Calidad de vida relacioada en salud.xlsx-------------- #
 # ---------------- Datos basicos.xlsx-------------- #
 # ---------------- Disnea.xlsx-------------- #
 # ---------------- Farmacovigilancia RAM.xlsx-------------- #
 # ---------------- Habitos.xlsx-------------- #
 # ---------------- Hospitalizaciones.xlsx-------------- #
 # ---------------- Incosistencias en reclamacion.xlsx-------------- #
 # ---------------- Medicamentos.xlsx-------------- #
 # ---------------- Mediciones de peso y talla.xlsx-------------- #
 # ---------------- Urgencias.xlsx-------------- #
 #

create table datos_basicos (
    patient_id integer primary key,
    name CHAR(55));

create hospitalizaciones (
    id integer primary key,
    country_id integer,
    name CHAR(55),
    foreign key (datos_basicos_patient_id) references datos_basicos(patient_id));
    
%%sql postgresql://
    \copy productmodelproductdescriptionculture FROM 'data/csvs/productmodelproductdescriptionculture.csv' with (format CSV, header true, delimiter ',


# To read an Excel file with multiple sheets

In [ ]:
#Read all sheets directly into an ordered dictionary.
sheet_to_df_map = pd.read_excel(file_name, sheet_name=None)

#Read the excel file and get a list of sheets. Then chose and load the sheets.
xls = pd.ExcelFile('excel_file_path.xls')
# Now you can list all sheets in the file
xls.sheet_names
# ['house', 'house_extra', ...]
# to read just one sheet to dataframe:
df = pd.read_excel(file_name, sheetname="house")